<a href="https://colab.research.google.com/github/seunghee0518/AI_python/blob/main/Outlier%20detection%20%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%20%EC%82%AC%EA%B8%B0%EA%B1%B0%EB%9E%98%20%EA%B0%90%EC%A7%80/%5Bbaseline2%5D%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EC%82%AC%EA%B8%B0_%EA%B1%B0%EB%9E%98_%ED%83%90%EC%A7%80_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C%5BPublic_score_0_93052%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [ ]:
import torch

import pandas as pd
import numpy as np

from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Load

In [ ]:
directory = '/content/drive/MyDrive/DACON_신용카드 사기 거래 탐지 AI 경진대회'
train_df = pd.read_csv(f'{directory}/data/train.csv') # Train
#train_df = train_df.drop(columns=['ID'])
#display(train_df.head())

val_df = pd.read_csv(f'{directory}/data/val.csv') # Validation
#val_df = val_df.drop(columns=['ID'])
#display(val_df.head())

test_df = pd.read_csv(f'{directory}/data/test.csv') # test
#display(test_df.head())

## Validation set 사기 거래 비율
#### (*) Validation set의 사기 거래 비율이 다른 데이터집합에서도 비슷하게 발생할 것이라고 가정

In [ ]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

Validation contamination : [0.0010551491277433877]


## Model Define & Fit

In [ ]:
# Train dataset은 Label이 존재하지 않음
train_x = train_df.drop(columns=['ID']) # Input Data

In [ ]:
# 가설 설정 : Train dataset도 Validation dataset과 동일한 비율로 사기거래가 발생 했을 것이다. -> model parameter : contamination=val_contamination(=0.001055) 적용
val_contamination = 0.00
model = EllipticEnvelope(support_fraction = 0.994, contamination = val_contamination, random_state = 42)
model.fit(train_x)

EllipticEnvelope(contamination=0.005, random_state=42, support_fraction=0.994)

In [ ]:
len(val_df[val_df['Class'] == 1])

30

k가 318인이유 : 
EllipticEnvelope의 predict 기능을 사용하여 제출할 경우 318개 보다 적은 개수를 사기로 잡아내는데 이 경우 퍼블릭 리더보드상의 점수가 낮더군요. 그래서 개수를 늘리다보니 315~318개에서 퍼블릭 리더보드 상 높은 점수를 얻었습니다 

# 0.005로 오염비율을 높여보았으나 k를 318ㄹ

In [ ]:
def get_pred_label(model, x, k):
  prob = model.score_samples(x) # 각 샘플의 로그 우도를 계산합니다.
  prob = torch.tensor(prob, dtype = torch.float) # 텐서로 변환 => n차원 배열 == 벡터
  topk_indices = torch.topk(prob, k = k, largest = False).indices # 결과값중 가장 큰값 k개만 이상치로 추출
  pred = torch.zeros(len(x), dtype = torch.long)
  pred[topk_indices] = 1
  return pred.tolist(), prob.tolist()

val_x = val_df.drop(columns=['ID', 'Class']) # Input Data
val_y = val_df['Class'] # Label

val_pred, val_prob = get_pred_label(model, val_x, 30) # vaild 데이터셋은 이상치가 30개임, 원본코드는 29로함
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))
tn, fp, fn, tp = confusion_matrix(val_y, val_pred).ravel()
print('tp : ', tp, ', fp : ', fp, ', tn : ', tn, ', fn : ', fn)

Validation F1 Score : [0.9165787375726882]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.83      0.83      0.83        30

    accuracy                           1.00     28462
   macro avg       0.92      0.92      0.92     28462
weighted avg       1.00      1.00      1.00     28462

tp :  25 , fp :  5 , tn :  28427 , fn :  5


## Inference : Test set

In [ ]:
# test_df = pd.read_csv('./test.csv') # Train
# test_df.head()

In [ ]:
test_x = test_df.drop(columns=['ID'])

In [ ]:
test_pred, _ = get_pred_label(model, test_x, 318)
print('n_fraud : ', sum(test_pred))

n_fraud :  318


## Submission

In [ ]:
submit = pd.read_csv(f'{directory}/data/sample_submission.csv')
submit.head()

,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [ ]:
submit['Class'] = test_pred
submit.to_csv(f'{directory}/submit/EllipticEnvelope_{val_contamination}_by_sh.csv', index=False)

In [ ]:
pd.read_csv('/content/submit_EllipticEnvelope.csv')['Class'].sum()

318